### Run logTgapsMetric on Multiple FBS_1.4 OpSims

Notebook uses Multiple_Opsims.ipynb from https://github.com/RichardsGroup/LSST_OpSim/tree/master/Scripts_NBs
as the template.

Here we analyze all 75 FBS 1.4 opSims in light of the distribution of time separations, which is already an existing metric [https://sims-maf.lsst.io/_modules/lsst/sims/maf/metrics/tgaps.html#TgapsMetric](https://sims-maf.lsst.io/_modules/lsst/sims/maf/metrics/tgaps.html#TgapsMetric).  However, in this case, we look at the distribution in *log* space.  Specifically from 30s to 10 years (converted to days).  The reasoning being that characterization of AGN structure functions (or PSDs) may benefit from a relatively uniform distribution in $log \Delta t$ space.  Rolling cadences may benefit AGNs (enabling more short time-separation observations), but extreme rolling cadences would be very bad (leaving few or no long time-separation observations).  For further discussion see the [relevant whitepaper](https://docushare.lsstcorp.org/docushare/dsweb/Get/Document-30572/richards_agn_rolling_wfd.pdf). 

### **Important:**  
In the next cell you need to update the `your_username` variable with **Your Username** (between the single quotes).  After you have done that, in principle, you should be able to run the notebook all at once instead of cell by cell. 

In [37]:
#Please enter your SciServer username between the single quotes below!
your_username = 'gtr'

In [38]:
# import matplotlib to show plots inline.
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os

Import the sims_maf modules needed.

In [39]:
# import lsst.sim.maf moduels modules
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.stackers as stackers
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metricBundles as metricBundles

### **Important:**  
The following code is needed before importing the `opsimUtils` module if the module is not in the same directory as the current notebook. That is, you need add the directory where the `opsimUtils` module is located to the Python search path. 

In [40]:
# add opsimUtils module path to search
import sys
sys.path.insert(0, '../Scripts_NBs/')

In [41]:
# import convenience functions
from opsimUtils import *

#### 1. Build connections to the OpSims databases
The first step is to initiate opsim database objects and result database objects for the opsim databases that you want to run metrics on. Two paths are needed here:
1. `dbDir`: The path to the OpSim database directory
2. `outDir`: The path to the directory where you want to save the metric metadata.

By providing these two paths, the function `connect_dbs()` will initiate connections and store the metadata to the designated directory.

In [42]:
if your_username == '': # do NOT put your username here, put it in the cell at the top of the notebook.
    raise Exception('Please provide your username!  See the top of the notebook.')

# Two path are needed
dbDir = '/home/idies/workspace/lsst_cadence/FBS_1.5/' #global location of opSims
#dbDir = '/home/idies/workspace/LSST Cadence Simulations/FBS_1.4/' #global location of opSims
outDir = '/home/idies/workspace/Temporary/{}/scratch/MAFoutput/DDFtest/'.format(your_username) #Output for GTR metrics

metricDataPath = '/home/idies/workspace/Temporary/{}/scratch/MAFoutput/DDFtest/MetricData/'.format(your_username)

if not os.path.exists(os.path.abspath(outDir)):
    os.mkdir(os.path.abspath(outDir))

In [43]:
# two dictionary are returned by the following function, 
# One (opSimDbs) is a dictionary storing all database objects
# Another (resultDbs) is a dictionary consist of the objects directing MAF where to save metric metadata
# Both dictionaries are indexed by OpSim run names
opSimDbs, resultDbs = connect_dbs(dbDir, outDir)

# Collect Run names to a list
dbRuns = list(opSimDbs.keys())

In [44]:
idx = [7,46,49,52]
dbRunsDDFtest = [dbRuns[i] for i in idx]
print(dbRunsDDFtest)

['daily_ddf_v1.5_10yrs', 'agnddf_v1.5_10yrs', 'baseline_v1.5_10yrs', 'descddf_v1.5_10yrs']


You can use `help` to get more information about the provided convenience functions, e.g., `connect_dbs()`

In [45]:
help(connect_dbs)

Help on function connect_dbs in module opsimUtils:

connect_dbs(dbDir, outDir)
    Initiate database objects to all opSim databases in the provided directory.
    Returns a dictionary consisting all database connections and a dictionary
    holding the resultsDb objects.
    
    Args:
        dbDir(str): The path to the dabase directory.
        outDir(str): The path to the result database directory.
    
    Returns:
        opSimDbs(dict): A dictionary containing the OpsimDatabase objects for
            opsim databases in the provided directory, keys are the run names.
        resultDbs(str): A dictionary containing the ResultsDb objects for opsim
            databases in the provided directory, keys are the run names.



You can also check what OpSims are available in the directory

In [46]:
show_opsims(dbDir)

['dcr_nham1_ugri_v1.5_10yrs',
 'rolling_mod6_sdf_0.20_v1.5_10yrs',
 'wfd_depth_scale0.95_noddf_v1.5_10yrs',
 'u60_v1.5_10yrs',
 'footprint_stuck_rollingv1.5_10yrs',
 'filterdist_indx4_v1.5_10yrs',
 'wfd_depth_scale0.70_noddf_v1.5_10yrs',
 'daily_ddf_v1.5_10yrs',
 'alt_roll_mod2_dust_sdf_0.20_v1.5_10yrs',
 'goodseeing_gz_v1.5_10yrs',
 'filterdist_indx1_v1.5_10yrs',
 'footprint_standard_goalsv1.5_10yrs',
 'footprint_big_sky_dustv1.5_10yrs',
 'footprint_bluer_footprintv1.5_10yrs',
 'twilight_neo_mod4_v1.5_10yrs',
 'goodseeing_gi_v1.5_10yrs',
 'footprint_big_wfdv1.5_10yrs',
 'dcr_nham2_ugr_v1.5_10yrs',
 'short_exp_5ns_5expt_v1.5_10yrs',
 'goodseeing_griz_v1.5_10yrs',
 'twilight_neo_mod1_v1.5_10yrs',
 'bulges_cadence_bulge_wfd_v1.5_10yrs',
 'third_obs_pt120v1.5_10yrs',
 'wfd_depth_scale0.95_v1.5_10yrs',
 'wfd_depth_scale0.85_noddf_v1.5_10yrs',
 'footprint_no_gp_northv1.5_10yrs',
 'footprint_big_skyv1.5_10yrs',
 'footprint_newBv1.5_10yrs',
 'baseline_2snaps_v1.5_10yrs',
 'footprint_big_sky_n

In [47]:
print(dbRuns[7])
print(dbRuns[46])
print(dbRuns[49])
print(dbRuns[52])

daily_ddf_v1.5_10yrs
agnddf_v1.5_10yrs
baseline_v1.5_10yrs
descddf_v1.5_10yrs


#### 2. Define logTgapsMetric

This is derived from https://sims-maf.lsst.io/lsst.sims.maf.metrics.html#module-lsst.sims.maf.metrics.tgaps

This metric is almost exactly the same, but hard codes the use of *all* pairs and bins that are log distributed (from 30s to 10 yrs).

In [48]:
#From 30s to 10 years (~3e8s), converted to days, 99 bins
bins=np.logspace(np.log10(30.0/60./60./24.),np.log10(3e8/60./60./24.),99)

This next cell defines the metric.  Note that all but one line below is meta data or bookkeeping and the one line that is the actual metric is a bit obtuse.  It might be useful to comment it in more detail (and perhaps to break it down into multiple lines that are easier to read, even if those lines are just commented out).

In [49]:
#from .baseMetric import BaseMetric
from lsst.sims.maf.metrics import BaseMetric

#__all__ = ['TgapsMetric', 'NightgapsMetric', 'NVisitsPerNightMetric']

class logTgapsMetric(BaseMetric):
    """Histogram the log of the times of the gaps between observations.

    equivalent to TgapsMetric, but with 
    allGaps=True 
    and 
    bins = np.logspace(-3.46,3.54,99) (30s to 10yrs, converted to days)
    
    That yields log distribution of time gaps for all pairs.

    Parameters
    ----------
    timesCol : str, opt
        The column name for the exposure times.  Values assumed to be in days.
        Default observationStartMJD.
    allGaps : bool, opt
        Histogram the gaps between all observations (True) or just successive observations (False)?
        Default is True. If all gaps are used, this metric can become significantly slower.
    bins : np.ndarray, opt
        The bins to use for the histogram of time gaps (in days, or same units as timesCol).
        Default values are log bins from 30s to 10 years.

    Returns a histogram at each slice point; these histograms can be combined and plotted using the
    'SummaryHistogram plotter'.
     """

    def __init__(self, timesCol='observationStartMJD', allGaps=True, \
                 bins=np.logspace(np.log10(30.0/60./60./24.),np.log10(3e8/60./60./24.),99), \
                 units='days', **kwargs):
        # Pass the same bins to the plotter.
        self.bins = bins
        self.timesCol = timesCol
        super(logTgapsMetric, self).__init__(col=[self.timesCol], metricDtype='object', units=units, **kwargs)
        self.allGaps = allGaps

    def run(self, dataSlice, slicePoint=None):
        if dataSlice.size < 2:
            return self.badval
        times = np.sort(dataSlice[self.timesCol])
        if self.allGaps:
            allDiffs = []
            for i in np.arange(1,times.size,1):
                allDiffs.append((times-np.roll(times,i))[i:])
            dts = np.concatenate(allDiffs)
        else:
            dts = np.diff(times)
        #print(dts,np.log10(dts))
        result, bins = np.histogram(dts, self.bins)
        #print(result)
        return result

#### 3. Declare metric to run on all the opSims.

Currently just looking at the logTgapsMetric in the r-band, but there is no reason not to do other bands (or to look at the DDFs).

# metric = analysis of histogram of log time separation for observations
metric1 = logTgapsMetric()

# slicer = a grouping or subdivision of visits for the simulated survey
# based on their position on the sky (using a Healpix grid)
slicer1 = slicers.HealpixSlicer(nside=64)

# constraint = the sql query (or 'select') that selects all visits in r band
#constraint1 = 'filter = "r"'
#constraint1 += ' and note like "DD%"' # added so the sky plot won't saturate (remove DDFs)

In [50]:
bands = ['u', 'g', 'r', 'i', 'z', 'y']
#ddfFields = ['COSMOS', 'XMM-LSS', 'ELAISS1', 'ECDFS', '290']
ddfFields = ['COSMOS', 'XMM-LSS', 'ELAISS1', 'ECDFS', 'EDFS']

# MetricBundle = combination of the metric, slicer, and sqlconstraint
logTgapsBundle = metricBundles.MetricBundle(metric1, slicer1, constraint1)

bundleDict = {'logTgaps': logTgapsBundle}

In [51]:
metric1 = logTgapsMetric()

In [52]:
logtgapsMetricNameTmp = 'logtgaps_{}_{}'
logtgapsSlicer = slicers.HealpixSlicer(nside=512)

Healpix slicer using NSIDE=512, approximate resolution 6.870973 arcminutes


In [53]:
cosmosSlicer = slicers.UserPointsSlicer(150.10833,2.233611)

In [54]:
# set summary statistics
summaryMetrics = [metrics.MinMetric(), metrics.MedianMetric(),
                  metrics.MaxMetric(), metrics.RmsMetric()]
# for metric in coaddBDict:
#     coaddBDict[metric].setSummaryMetrics(summaryMetrics)

In [56]:
# loop through opsims
for run in dbRunsDDFtest:
    print(f'Runing metrics on Opsim: {run}!')
    print('*****************************************')
    # one metric bundle dict per opsim
    logtgapsBDict = {}
    
    # loop through each DDF to get proposalId
    for ddf in ddfFields[0:1]: #[0:5]
        propInfo = ddfInfo(opSimDbs[run], ddf)
        
        if propInfo is None:
            print(f'Skip Opsim {run}!')
            break
        else:
            for band in bands[1:4]: #[1:4]
                metricName = logtgapsMetricNameTmp.format(ddf, band)
                print(metricName)
                #logtgapsMetric = metrics.CountMetric('observationStartMJD', metricName=metricName)
                #logtgapsMetric = metrics.CountMetric('observationStartMJD', metricName=metricName)
                logtgapsMetric =  logTgapsMetric(metricName=metricName)
                logtgapsConstraint = 'filter = "{}"'.format(band)
                logtgapsConstraint += ' and proposalId = {}'.format(propInfo['proposalId'])
                logtgapsBDict[metricName] = metricBundles.MetricBundle(logtgapsMetric, \
                                        cosmosSlicer, logtgapsConstraint, runName=run)
                #logtgapsBDict[metricName].setSummaryMetrics(summaryMetrics)

    metricGroup = metricBundles.MetricBundleGroup(logtgapsBDict, opSimDbs[run], \
                                metricDataPath, resultDbs[run], verbose=False)
    metricGroup.runAll()

Runing metrics on Opsim: daily_ddf_v1.5_10yrs!
*****************************************
logtgaps_COSMOS_g
logtgaps_COSMOS_r
logtgaps_COSMOS_i
Runing metrics on Opsim: agnddf_v1.5_10yrs!
*****************************************
logtgaps_COSMOS_g
logtgaps_COSMOS_r
logtgaps_COSMOS_i
Runing metrics on Opsim: baseline_v1.5_10yrs!
*****************************************
logtgaps_COSMOS_g
logtgaps_COSMOS_r
logtgaps_COSMOS_i
Runing metrics on Opsim: descddf_v1.5_10yrs!
*****************************************
logtgaps_COSMOS_g
logtgaps_COSMOS_r
logtgaps_COSMOS_i


#### 4. Loop over all OpSims in dbDir and run MAF
While constructing a metricBundleGroup from a dictionary (the cell below), you will need to provide the path to a directory (`metricDataPath` in the cell below) where you would like to store the metric data (this is **DIFFERENT** than path to the metric data, `outDir`). To construct metricbundles for plotting and further analysis, this path will be needed.

metricDataPath = '/home/idies/workspace/Storage/gtr/persistent/MAFOutput/MetricData/'
#for run in dbRuns[0:3]:  #To just run the first few.
for i,run in enumerate(dbRuns):
    # must set run name for each opSim to store metric data into
    # separate files
    print(i,run) #Comment out if just running the first few
    logTgapsBundle.setRunName(run)
    metricGroup = metricBundles.MetricBundleGroup(bundleDict,\
                    opSimDbs[run], metricDataPath, resultDbs[run])
    metricGroup.runAll()

Once this script has finished (about 5 hours), you can make plots with [plotLogTgapsMetric.ipynb](01_plotLogTgapsMetric.ipynb).